## tensorflow version

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from sionna.utils import expand_to_rank

2024-06-29 11:31:32.898536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 11:31:32.898571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 11:31:32.916038: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-29 11:31:32.954766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 11:31:34.425510: W tensorflow/compiler/tf2

# BinaryMemorylessChannel类

`BinaryMemorylessChannel`类是一个离散的二进制无记忆通道，具有（可能的）不对称比特翻转概率。

输入的比特以概率 \( p_\text{b,0} \) 和 \( p_\text{b,1} \) 分别翻转。

这个层支持二进制输入（\( x \in \{0, 1\} \)）和双极性输入（\( x \in \{-1, 1\} \)）。

如果激活，该通道直接返回对数似然比（LLRs），定义如下：

\[
\ell =
\begin{cases}
    \operatorname{log} \frac{p_{b,1}}{1-p_{b,0}}, \qquad \text{如果} \, y=0 \\
    \operatorname{log} \frac{1-p_{b,1}}{p_{b,0}}, \qquad \text{如果} \, y=1 \\
\end{cases}
\]

错误概率 \( p_\text{b}\) 可以是标量或张量（可广播到输入的形状）。这允许每个位位置有不同的擦除概率。在任何情况下，它的最后一个维度的长度必须为2，并被解释为 \( p_\text{b,0} \) 和 \( p_\text{b,1} \)。

这个类继承自Keras的`Layer`类，可以在Keras模型中用作层。

## 参数

- `return_llrs`：布尔类型，默认值为 `False`。如果设置为 `True`，则该层返回基于 `pb` 的对数似然比（LLRs）而不是二进制值。

- `bipolar_input`：布尔类型，默认值为 `False`。如果设置为 `True`，则期望的输入是 \( \{-1,1\} \) 而不是 \( \{0,1\} \)。

- `llr_max`：`tf.float`类型，默认值为100。定义LLRs的裁剪值。

- `dtype`：`tf.DType`类型，定义内部计算和输出的类型。默认值为 `tf.float32`。

## 输入

- `(x, pb)`：
  - `x`：形状为 [...,n] 的 `tf.float32` 类型的输入序列，包含二进制值 \( \{0,1\} \) 或 \( \{-1,1\} \)。
  - `pb`：形状为 [...,2] 的 `tf.float32` 类型的错误概率。可以是两个标量的元组或任何可以广播到 `x` 形状的形状。它有一个额外的最后维度，解释为 \( p_\text{b,0} \) 和 \( p_\text{b,1} \)。

## 输出

- 形状为 [...,n] 的 `tf.float32` 类型的输出序列，长度与输入 `x` 相同。如果 `return_llrs` 为 `False`，则输出是三元的，其中 `-1` 和 `0` 分别表示二进制和双极性输入的擦除。

In [2]:
class BinaryMemorylessChannel(Layer):
    def __init__(self, return_llrs=False, bipolar_input=False, llr_max=100.,dtype=tf.float32, **kwargs):

        super().__init__(dtype=dtype,**kwargs)

        assert isinstance(return_llrs, bool), "return_llrs must be bool."
        self._return_llrs = return_llrs

        assert isinstance(bipolar_input, bool), "bipolar_input must be bool."
        self._bipolar_input = bipolar_input

        assert llr_max>=0., "llr_max must be a positive scalar value."
        self._llr_max = tf.cast(llr_max, dtype=self.dtype)

        if self._return_llrs:
            assert dtype in (tf.float16, tf.float32, tf.float64),\
                "LLR outputs require non-integer dtypes."
        else:
            if self._bipolar_input:
                assert dtype in (tf.float16, tf.float32, tf.float64,
                    tf.int8, tf.int16, tf.int32, tf.int64),\
                    "Only, signed dtypes are supported for bipolar inputs."
            else:
                assert dtype in (tf.float16, tf.float32, tf.float64,
                    tf.uint8, tf.uint16, tf.uint32, tf.uint64,
                    tf.int8, tf.int16, tf.int32, tf.int64),\
                    "Only, real-valued dtypes are supported."

        self._check_input = True # check input for consistency (i.e., binary)

        self._eps = 1e-9 # small additional term for numerical stability
        self._temperature = tf.constant(0.1, tf.float32) # for Gumble-softmax


在这个`BinaryMemorylessChannel`类的构造函数中，有多个步骤来初始化和验证输入参数。下面是对每一行代码的详细解释：

```python
def __init__(self, return_llrs=False, bipolar_input=False, llr_max=100., dtype=tf.float32, **kwargs):
```

- 这是类的构造函数（初始化方法），用于在创建类的实例时设置初始值。
- `return_llrs`: 是否返回对数似然比 (LLRs)，默认值为`False`。
- `bipolar_input`: 输入是否为双极性的（{-1, 1}），默认值为`False`。
- `llr_max`: LLRs的最大剪辑值，默认值为`100`。
- `dtype`: 数据类型，默认值为`tf.float32`。
- `**kwargs`: 其他可选参数。

```python
super().__init__(dtype=dtype, **kwargs)
```

- 调用父类`Layer`的初始化方法，并传递数据类型和其他参数。

```python
assert isinstance(return_llrs, bool), "return_llrs must be bool."
self._return_llrs = return_llrs
```

- 检查`return_llrs`是否为布尔值，如果不是，抛出一个断言错误。
- 将`return_llrs`保存为实例变量`self._return_llrs`。

```python
assert isinstance(bipolar_input, bool), "bipolar_input must be bool."
self._bipolar_input = bipolar_input
```

- 检查`bipolar_input`是否为布尔值，如果不是，抛出一个断言错误。
- 将`bipolar_input`保存为实例变量`self._bipolar_input`。

```python
assert llr_max >= 0., "llr_max must be a positive scalar value."
self._llr_max = tf.cast(llr_max, dtype=self.dtype)
```

- 检查`llr_max`是否为非负值，如果不是，抛出一个断言错误。
- 将`llr_max`转换为指定的数据类型并保存为实例变量`self._llr_max`。

```python
if self._return_llrs:
    assert dtype in (tf.float16, tf.float32, tf.float64), "LLR outputs require non-integer dtypes."
```

- 如果返回LLRs，则检查数据类型是否为`tf.float16`、`tf.float32`或`tf.float64`中的一种。如果不是，抛出一个断言错误。

```python
else:
    if self._bipolar_input:
        assert dtype in (tf.float16, tf.float32, tf.float64, tf.int8, tf.int16, tf.int32, tf.int64),\
            "Only, signed dtypes are supported for bipolar inputs."
```

- 如果不返回LLRs，且输入为双极性，则检查数据类型是否为`tf.float16`、`tf.float32`、`tf.float64`、`tf.int8`、`tf.int16`、`tf.int32`或`tf.int64`中的一种。如果不是，抛出一个断言错误。

```python
    else:
        assert dtype in (tf.float16, tf.float32, tf.float64, tf.uint8, tf.uint16, tf.uint32, tf.uint64, tf.int8, tf.int16, tf.int32, tf.int64),\
            "Only, real-valued dtypes are supported."
```

- 如果不返回LLRs，且输入不是双极性，则检查数据类型是否为`tf.float16`、`tf.float32`、`tf.float64`、`tf.uint8`、`tf.uint16`、`tf.uint32`、`tf.uint64`、`tf.int8`、`tf.int16`、`tf.int32`或`tf.int64`中的一种。如果不是，抛出一个断言错误。

```python
self._check_input = True # check input for consistency (i.e., binary)
```

- 设置一个实例变量`self._check_input`为`True`，用于在后续方法中检查输入的一致性（即是否为二进制）。

```python
self._eps = 1e-9 # small additional term for numerical stability
```

- 设置一个实例变量`self._eps`为`1e-9`，用于数值稳定性的小附加项。

```python
self._temperature = tf.constant(0.1, tf.float32) # for Gumble-softmax
```

- 设置一个实例变量`self._temperature`为常量`0.1`，用于Gumbel-Softmax。

### 详细解释

通过这些初始化步骤，`BinaryMemorylessChannel`类的构造函数确保了所有输入参数都是有效的，并根据这些参数设置了一些实例变量，用于后续的计算和操作。这些检查和设置有助于提高代码的健壮性和可维护性。

### 详细解释 `Gumbel-Softmax` 机制

`Gumbel-Softmax` 是一种用于从离散分布中采样的近似方法，特别是在需要梯度信息的情况下。这在机器学习和深度学习中非常有用，例如在生成模型或离散动作空间的强化学习中。

#### Gumbel 分布

首先，`Gumbel-Softmax` 的基础是 Gumbel 分布。Gumbel 分布是一种极值分布，通常用于建模最大值或最小值的分布。其概率密度函数 (PDF) 为：

$$
f(x) = e^{-(x + e^{-x})}
$$

Gumbel 分布的一个重要性质是其用于最大值采样时的重参数化技巧。

#### Gumbel-Max 采样

给定一个类别分布 \(\pi = (\pi_1, \pi_2, \ldots, \pi_k)\)，我们可以使用 Gumbel-Max 采样方法来从中采样一个类别：

$$
\text{sample} = \arg\max_i \left( \log(\pi_i) + g_i \right)
$$

其中 $(g_i)$是从 Gumbel(0,1) 分布中采样的随机变量。

#### Gumbel-Softmax

然而，Gumbel-Max 采样是离散的，不适合梯度优化。因此，Gumbel-Softmax 引入了一个温度参数 $(\tau)$，并将采样过程近似为：

$$
y_i = \frac{\exp\left((\log(\pi_i) + g_i) / \tau \right)}{\sum_{j=1}^k \exp\left((\log(\pi_j) + g_j) / \tau \right)}
$$

当 $(\tau \to 0)$ 时，Gumbel-Softmax 分布趋近于离散分布。当 $(\tau \to \infty)$ 时，分布变得更加均匀。通过调整 $(\tau)$，我们可以控制采样的离散程度。

### `Gumbel-Softmax` 在 `BinaryMemorylessChannel` 中的应用

在 `BinaryMemorylessChannel` 类的初始化过程中，设置 `self._temperature = tf.constant(0.1, tf.float32)` 是为了在某些计算中使用 `Gumbel-Softmax` 机制。具体来说，`Gumbel-Softmax` 可以用于模拟离散的二元信道，同时保持对梯度的支持。这对于神经网络的训练非常重要，因为它允许使用反向传播来优化参数。

### 总结

通过上述初始化和检查步骤，`BinaryMemorylessChannel` 类确保输入参数有效，并设置实例变量以支持后续的计算和操作。其中，`Gumbel-Softmax` 机制的引入，使得在处理离散信道采样时，能够有效地进行梯度优化，从而提高模型的性能和训练效率。

# BinaryMemorylessChannel (torch version)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from my_code.mysionna.channel.torch_version.utils import expand_to_rank


class CustomOperations:
    
    class CustomXOR(Function):
        @staticmethod
        def forward(ctx, a, b):
            ctx.save_for_backward(a, b)
            if a.dtype in (torch.uint8, torch.int8, torch.int16, torch.int32, torch.int64):
                z = (a + b) % 2
            else:
                z = torch.abs(a - b)
            return z

        @staticmethod
        def backward(ctx, grad_output):
            return grad_output, grad_output
    # STEBinarizer（Straight-Through Estimator Binarizer）是一种在神经网络中用于处理二值化操作的技术。
    # STE代表Straight-Through Estimator，它是一种用于在反向传播中处理不可微操作的技术。
    class STEBinarizer(Function):
        @staticmethod
        def forward(ctx, x):
            ctx.save_for_backward(x)
            z = torch.where(x < 0.5, torch.tensor(0.0, device=x.device), torch.tensor(1.0, device=x.device))
            return z

        @staticmethod
        def backward(ctx, grad_output):
            return grad_output
    
    class SampleErrors(torch.nn.Module):
        def __init__(self, eps=1e-10, temperature=1.0):
            super().__init__()
            self._eps = eps
            self._temperature = temperature

        def forward(self, pb, shape):
            u1 = torch.rand(shape, dtype=torch.float32)
            u2 = torch.rand(shape, dtype=torch.float32)
            u = torch.stack((u1, u2), dim=-1)

            # 采样Gumbel分布
            q = -torch.log(-torch.log(u + self._eps) + self._eps)
            p = torch.stack((pb, 1 - pb), dim=-1).unsqueeze(1).expand(shape[0], shape[1], 2)
            a = (torch.log(p + self._eps) + q) / self._temperature

            # 应用softmax
            e_cat = F.softmax(a, dim=-1)

            # 通过直通估计器对最终值进行二值化
            return CustomOperations.STEBinarizer.apply(e_cat[..., 0])

class BinaryMemorylessChannel(nn.Module):
    def __init__(self, return_llrs=False, bipolar_input=False, llr_max=100., dtype=torch.float32, **kwargs):
        super().__init__(**kwargs)

        assert isinstance(return_llrs, bool), "return_llrs must be bool."
        self.return_llrs = return_llrs

        assert isinstance(bipolar_input, bool), "bipolar_input must be bool."
        self.bipolar_input = bipolar_input

        assert llr_max >= 0., "llr_max must be a positive scalar value."
        self.llr_max = llr_max
        self.dtype = dtype

        if self.return_llrs:
            assert dtype in (torch.float16, torch.float32, torch.float64), \
                "LLR outputs require non-integer dtypes."
        else:
            if self.bipolar_input:
                assert dtype in (torch.float16, torch.float32, torch.float64,
                                 torch.int8, torch.int16, torch.int32, torch.int64), \
                    "Only signed dtypes are supported for bipolar inputs."
            else:
                assert dtype in (torch.float16, torch.float32, torch.float64,
                                 torch.uint8, torch.uint16, torch.uint32, torch.uint64,
                                 torch.int8, torch.int16, torch.int32, torch.int64), \
                    "Only real-valued dtypes are supported."

        self.check_input = True  # check input for consistency (i.e., binary)

        self.eps = 1e-9  # small additional term for numerical stability
        self.temperature = torch.tensor(0.1, dtype=torch.float32)  # for Gumble-softmax

    @property
    def llr_max(self):
        """Maximum value used for LLR calculations."""
        return self._llr_max

    @llr_max.setter
    def llr_max(self, value):
        """Maximum value used for LLR calculations."""
        assert value >= 0, 'llr_max cannot be negative.'
        self._llr_max = value

    @property
    def temperature(self):
        """Temperature for Gumble-softmax trick."""
        return self._temperature.item()

    @temperature.setter
    def temperature(self, value):
        """Temperature for Gumble-softmax trick."""
        assert value >= 0, 'temperature cannot be negative.'
        self._temperature = torch.tensor(value, dtype=torch.float32)
    #########################
    # Utility methods
    #########################

    def _check_inputs(self, x):
        """Check input x for consistency, i.e., verify
        that all values are binary of bipolar values."""
        x = x.float()
        if self.check_input:
            if self.bipolar_input:
                assert torch.all(torch.logical_or(x == -1, x == 1)), "Input must be bipolar {-1, 1}."
            else:
                assert torch.all(torch.logical_or(x == 0, x == 1)), "Input must be binary {0, 1}."
            # input datatype consistency should be only evaluated once
            self.check_input = False

    # 使用方法
    @staticmethod
    def custom_xor(a, b):
        return CustomOperations.CustomXOR.apply(a, b)       

    @staticmethod
    def ste_binarizer(self, x):
        """Straight through binarizer to quantize bits to int values."""
        return CustomOperations.STEBinarizer.apply(x)

    def _sample_errors(self, pb, shape):
        """Samples binary error vector with given error probability e.
        This function is based on the Gumble-softmax "trick" to keep the
        sampling differentiable."""

        u1 = torch.rand(shape)
        u2 = torch.rand(shape)
        u = torch.stack((u1, u2), dim=-1)

        # sample Gumble distribution
        q = -torch.log(-torch.log(u + self.eps) + self.eps)
        p = torch.stack((pb, 1 - pb), dim=-1)
        p = p.unsqueeze(0).expand(q.shape)
        a = (torch.log(p + self.eps) + q) / self.temperature

        # apply softmax
        e_cat = F.softmax(a, dim=-1)

        # binarize final values via straight-through estimator
        return self._ste_binarizer(e_cat[..., 0])  # only take the first class
    
    #########################
    # Keras layer functions
    #########################

    # 这段代码定义了一个 build 方法了，用于验证输入的形状是否正确
    # 它主要检查第二个输入（错误概率 pb）的形状，确保其最后一维的长度为 2

    def build(self, input_shapes):
        """Verify correct input shapes"""

        pb_shapes = input_shapes[1]
        # allow tuple of scalars as alternative input
        if isinstance(pb_shapes, (tuple, list)):
            if not len(pb_shapes) == 2:
                raise ValueError("Last dim of pb must be of length 2.")
        else:
            if len(pb_shapes) > 0:
                if not pb_shapes[-1] == 2:
                    raise ValueError("Last dim of pb must be of length 2.")
            else:
                raise ValueError("Last dim of pb must be of length 2.")
            
    def call(self, inputs):
        """Apply discrete binary memoryless channel to inputs."""

        x, pb = inputs

        # allow pb to be a tuple of two scalars
        if isinstance(pb, (tuple, list)):
            pb0 = pb[0]
            pb1 = pb[1]
        else:
            pb0 = pb[...,0]
            pb1 = pb[...,1]
        
        # 假设pb0和pb1是PyTorch张量
        pb0 = pb0.float()  # 确保pb0是浮点数
        pb1 = pb1.float()  # 确保pb1是浮点数
        pb0 = torch.clamp(pb0, 0., 1.)  # 将pb0的值限制在0和1之间
        pb1 = torch.clamp(pb1, 0., 1.)  # 将pb1的值限制在0和1之间

        # check x for consistency (binary, bipolar)
        self._check_inputs(x)

        e0 = self._sample_errors(pb0,x.shape)
        e1 = self._sample_errors(pb1, x.shape)

        if self._bipolar_input:
            neutral_element = torch.tensor(-1, dtype=x.dtype)
        else:
            neutral_element = torch.tensor(0, dtype=x.dtype)    

        # mask e0 and e1 with input such that e0 only applies where x==0    
        e = torch.where(x == neutral_element, e0, e1)
        e = e.to(dtype=x.dtype)

        if self._bipolar_input:
            # flip signs for bipolar case
            y = x * (-2*e + 1)
        else:
            # XOR for binary case
            y = self.custom_xor(x, e)

        # if LLRs should be returned
        if self._return_llrs:
            if not self._bipolar_input:
                y = 2 * y - 1  # transform to bipolar
            # Remark: Sionna uses the logit definition log[p(x=1)/p(x=0)]
            # 计算LLRs的组成部分
            y0 = -(torch.log(pb1 + self._eps) - torch.log(1 - pb0 - self._eps))
            y1 = (torch.log(1 - pb1 - self._eps) - torch.log(pb0 + self._eps))

            # multiply by y to keep gradient
            # 使用torch.where实现条件选择
            y = torch.where(y == 1, y1, y0).to(dtype=y.dtype) * y

            # and clip output llrs
            # 将LLR的值限制在范围内
            y = torch.clamp(y, min=-self._llr_max, max=self._llr_max)        

        return y

"""这段代码定义了一个PyTorch模块`CustomOperations`，其中包含几个用于自定义操作的类和方法。以下是中文表述：

### 类 `CustomOperations`：
- 包含几个用于执行特定数学操作的内部类，包括`CustomXOR`和`STEBinarizer`，以及一个用于采样二元错误向量的模块`SampleErrors`。

#### 类 `CustomXOR`（继承自`torch.autograd.Function`）：
- 用于执行自定义的异或操作，支持整数和浮点数数据类型。
- `forward`方法：执行异或操作，如果是整数类型则使用模2运算，否则使用绝对差值。
- `backward`方法：在反向传播中，将梯度直接传回给输入。

#### 类 `STEBinarizer`（继承自`torch.autograd.Function`）：
- 用于实现Straight-Through Estimator Binarizer，即直通估计器二值化。
- `forward`方法：在前向传播中，使用`torch.where`实现阈值化操作，将小于0.5的值设为0，否则设为1。
- `backward`方法：在反向传播中，将梯度直接传回（直通估计器）。

#### 类 `SampleErrors`（继承自`torch.nn.Module`）：
- 用于根据给定的错误概率`pb`和形状`shape`采样二元错误向量。
- 在初始化中接受一个小的正则化项`eps`和温度参数`temperature`。
- `forward`方法：首先生成Gumbel分布的样本，然后通过softmax函数和直通估计器二值化来模拟二元错误。

#### 类 `BinaryMemorylessChannel`（继承自`torch.nn.Module`）：
- 用于模拟离散的二元对称信道，可以随机翻转比特位，翻转概率为`p_b`。
- 接受是否返回LLRs的标志`return_llrs`，是否接受双极输入的标志`bipolar_input`，LLRs的最大值`llr_max`，数据类型`dtype`等参数。
- 包含属性和方法用于设置和获取`llr_max`和`temperature`。
- `_check_inputs`方法：检查输入`x`是否为二元或双极值。
- `custom_xor`静态方法：调用`CustomXOR.apply`执行异或操作。
- `ste_binarizer`静态方法：调用`STEBinarizer.apply`实现二值化。
- `_sample_errors`方法：根据Gumbel-Softmax技巧采样二元错误向量。
- `build`方法：验证输入形状是否正确。
- `call`方法：应用离散二元对称信道到输入。

在`call`方法中，根据是否返回LLRs、是否双极输入，以及给定的翻转概率`pb`，执行不同的操作来生成输出。如果需要返回LLRs，将输出转换为LLRs，并通过`torch.clamp`方法限制其值在`[-llr_max, llr_max]`范围内。

整体上，这段代码提供了一个灵活的框架，用于在PyTorch中实现和使用自定义的神经网络操作，特别是那些涉及二值化和异或操作的场景。
"""

## 关于STE
[pytorch实现简单的straight-through estimator(STE)](https://segmentfault.com/a/1190000020993594)

现在深度学习中一般我们学习的参数都是连续的，因为这样在反向传播的时候才可以对梯度进行更新。但是有的时候我们也会遇到参数是离>散的情况，这样就没有办法进行反向传播了，比如二值神经网络。本文中讲解了如何用`pytorch`对二值化的参数进行梯度更新的`straight-through estimator`算法。
### Question
`STE`核心的思想就是我们的参数初始化的时候就是`float`这样的连续值，当我们`forward`的时候就将原来的连续的参数映射到{-1,, 1}带入到网络进行计算，这样就可以计算网络的输出。然后`backward`的时候直接对原来`float`的参数进行更新，而不是对二值化的参数更新。这样可以完成对整个网络的更新了。
首先我们对上面问题进行一下数学的讲解。

- 我们希望参数的范围是$r \in \mathbb{R}$
- 我们可以得到二值化的参数 $q = Sign(r)$, $Sign$函数可以参考`torch.sign`函数, 可以理解为取符号函数
- `backward`的过程中对$q$求梯度可得 $\frac{\partial loss}{\partial q}$
- 对于$\frac{\partial q}{\partial r} = 0$, 所以可以得出 $\frac{\partial loss}{\partial r} = 0$, 这样的话我们就无法完成对参>数的更新，因为每次`loss`对`r`梯度都是0
- 所以`backward`的过程我们需要修改$\frac{\partial q}{\partial r}$这部分才可以使梯度继续更新下去，所以对$\frac{\partial loss}{\partial r}$进行如下修改: $\frac{\partial q}{\partial r} = \frac{\partial loss}{\partial q} * 1\_{|r| \leq 1}$, 其中
$1\_{|r| \leq 1}$ 可以看作$Htanh(x) = Clip(x, -1, 1) = max(-1, min(1, x))$对$x$的求导过程, 也就是是说:
$$ \frac{\partial loss}{\partial r} =  \frac{\partial loss}{\partial q} \frac{\partial Htanh}{\partial r}$$

### Example
#### torch.sign
首先我们验证一下使用`torch.sign`会是参数的梯度基本上都是0:

In [6]:
input = torch.randn(4, requires_grad = True)
output = torch.sign(input)
loss = output.mean()
loss.backward()
print(input)
print(input.grad)

tensor([ 0.1455,  0.7444,  0.9377, -0.7715], requires_grad=True)
tensor([0., 0., 0., 0.])


#### demo
我们需要重写sign这个函数，就好像写一个激活函数一样。先看一下代码：

In [7]:
import torch

class LBSign(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        return torch.sign(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clamp_(-1, 1)

接下来我们做一下测试

In [9]:
sign = LBSign.apply
params = torch.randn(4, requires_grad = True)                                                                           
output = sign(params)
loss = output.mean()
loss.backward()
print(params)
print(params.grad)

tensor([ 1.1393, -0.2865,  1.7476, -0.3908], requires_grad=True)
tensor([0.2500, 0.2500, 0.2500, 0.2500])
